# Reading and analysing data from Yahoo Finance

Taking the functions from utils.py, to make the code more modular.

In [ ]:
from utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
# logging level: NOTSET, DEBUG, INFO, WARNING, ERROR, CRITICAL
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
INPUT_FOLDER_NAME_INFO = "/Users/abuzatu/Work/data/finance/stocks/info"
INPUT_FILE_NAME = "/Users/abuzatu/Work/data/finance/stocks/tickers1.txt"
INPUT_FOLDER_NAME = "/Users/abuzatu/Work/data/finance/stocks/processed_data"
OUTPUT_FOLDER_NAME = "/Users/abuzatu/Work/data/finance/stocks/processed_data/210111"

# creates output folder
Path(OUTPUT_FOLDER_NAME).mkdir(parents=True, exist_ok=True)

In [ ]:
PERIOD = None
ADD_OUTSIDE_TRADING_HOURS = True
ADD_DIVIDENDS_AND_STOCK_SPLITS = True
AUTO_ADJUST = True


In [ ]:
if True:
    LIST_DATE = [
        (pd.to_datetime("2021-01-04"), pd.to_datetime("2021-01-11"), "1m")
    ]
else:
    LIST_DATE = None
LIST_DATE

In [ ]:
list_stock_ticker_folder = get_list_stock_ticker_in_folder(INPUT_FOLDER_NAME)
list_stock_ticker_folder

In [ ]:
list_stock_ticker_file = get_list_stock_ticker_from_file(INPUT_FILE_NAME)
list_stock_ticker_file

In [ ]:
df = run_all_stock_stickers(INPUT_FOLDER_NAME_INFO, INPUT_FOLDER_NAME, INPUT_FILE_NAME, OUTPUT_FOLDER_NAME, ADD_OUTSIDE_TRADING_HOURS, ADD_DIVIDENDS_AND_STOCK_SPLITS, AUTO_ADJUST)

In [ ]:
df = pd.read_pickle("/Users/abuzatu/Work/data/finance/stocks/processed_data/210102/df_20-12-04_20-12-11_1m_TSLA.pickle")
df

In [ ]:
df1 = pd.read_pickle("/Users/abuzatu/Work/data/finance/stocks/processed_data/210102/df_20-12-11_20-12-18_1m_TSLA.pickle")
df1

In [ ]:
df2 = pd.read_pickle("/Users/abuzatu/Work/data/finance/stocks/processed_data/210109/df_2020-12-12_2020-12-19_1m_TSLA.pickle")
df2

In [ ]:
path = Path(f"{INPUT_FOLDER_NAME_INFO}/info.pickle")
path

In [ ]:
path.exists()

In [ ]:
df

In [ ]:
df = pd.DataFrame([], columns = ["stock_ticker", "date_first"])
df.set_index("stock_ticker", inplace = True)
df

In [ ]:
df.loc["ENG"]["date_first"]

In [ ]:
df.to_pickle(f"{INPUT_FOLDER_NAME_INFO}/info.pickle")

In [ ]:
df = pd.read_pickle(f"{INPUT_FOLDER_NAME_INFO}/info.pickle")
df

In [ ]:
"ENG2" in df.index

In [ ]:
# add a new element
df.loc["ENG"] = "d"

In [ ]:
df.loc["BLA"] = "b"

In [ ]:
df

In [ ]:
df.loc[len(df.index)]=[]

In [ ]:
df.reset_index(inplace = True)
df.append({"date_first" : "2020-08-01"}, index = "NIO", ignore_index=True, sort=False)
df

In [ ]:
assert False

In [ ]:
# read the file and save to a file
str_today = str(pd.Timestamp.today().date())
logging.info(f"Today is {str_today}")
for stock_ticker in list_stock_ticker_file:
    
    if False:
        # do only for one ticker
        if stock_ticker != "ZOM":
            continue
    already_have = stock_ticker in list_stock_ticker
    logging.info(f"stock_ticker={stock_ticker}, already_have={already_have}")
    
    continue
    
    # create list of dates
    if LIST_DATE is None:
        # find automatically the range that we want
        # we collect the data using the period max, then find the first date
        # than depending on that date build the LIST_DATE
        df = read_data(stock_ticker,
                   "max",
                   None,
                   None,
                   "1d",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
    
        # calculate list_date
        str_date_start = str(df.index[0].date())
        str_date_end = str_today
        logging.info(f"str_date_start={str_date_start}, str_date_end={str_date_end}")
        list_date = get_list_date(str_date_start, str_date_end)
    else:
        list_date = LIST_DATE
    for date in list_date:
        logging.info(f"{date}")
        
    do_store_all_period_to_file = False
    do_store_only_some_period_to_file = False
    df = get_df_from_list_date(stock_ticker, list_date, OUTPUT_FOLDER_NAME, PERIOD, ADD_OUTSIDE_TRADING_HOURS,  ADD_DIVIDENDS_AND_STOCK_SPLITS, AUTO_ADJUST, do_store_all_period_to_file, do_store_only_some_period_to_file)
    
    do_combine_with_previous_file = True   
    if do_combine_with_previous_file:
        logging.debug("Start do_combine_with_previous_file.")
        # retrieve the previous file
        date_start_all = "2019-01-01"
        date_end_all = "2021-01-01"
        suffix = "al"
        input_file_name_data = f"{OUTPUT_FOLDER_NAME}/df_{date_start_all}_{date_end_all}_{suffix}_{stock_ticker}.pickle"
        df__ = pd.read_pickle(input_file_name_data)
        logging.debug(f"Previous file {input_file_name_data} has {len(df__)} entries.")
    
        logging.debug("Start concatenating the old and new file")
        df_ = pd.concat([df__, df], axis = 0)
        logging.debug("Sort by index in chronolical order")
        df_.sort_index(inplace = True)
        logging.debug("Remove potential rare wrong values for after hours that are just to huge.")
        df_ = df_[df_.Close < 1e6] # eliminate sometimes very high wrong values
    
        # write to file
        date_end_all = str(LIST_DATE[-1][1].date())
        output_file_name_data = f"{OUTPUT_FOLDER_NAME}/df_{date_start_all}_{date_end_all}_{suffix}_{stock_ticker}.pickle"
        df_.to_pickle(output_file_name_data)
        logging.debug(f"Written final df_ with {len(df_)} elements to file {output_file_name_data}.")
    

In [ ]:
assert False

In [ ]:
stock_ticker =  "ENG"
df1 = read_data(stock_ticker,
                   "max",
                   None,
                   None,
                   "1d",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
df1

In [ ]:
df2 = df1[df1.index > pd.to_datetime("2020-06-01").tz_localize(LOCALIZE_US_STOCK_MARKET)]
df2

In [ ]:
plt.plot(df2.Close)

In [ ]:
df2 = read_data(stock_ticker,
                   None,
                   pd.to_datetime("2020-11-11").tz_localize(LOCALIZE_US_STOCK_MARKET),
                   pd.to_datetime("2020-12-11").tz_localize(LOCALIZE_US_STOCK_MARKET),
                   "5m",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
df2

In [ ]:
df2.to_pickle("/Users/abuzatu/Work/data/finance/stocks/processed_data/210109/df_2020-11-11_2020-12-11_5m_ENG.pickle")

In [ ]:
df22 = df2.copy()
df22

In [ ]:
df2 = read_data(stock_ticker,
                   None,
                   pd.to_datetime("2020-11-11"),#.tz_localize("Europe/Berlin"),
                   pd.to_datetime("2020-12-11"),#.tz_localize("Europe/Berlin"),
                   "5m",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
df2

In [ ]:
df32 = read_data(stock_ticker,
                   None,
                   pd.to_datetime("2020-11-11"),#.tz_localize("Europe/Berlin"),
                   pd.to_datetime("2020-12-11"),#.tz_localize("Europe/Berlin"),
                   "5m",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
df32

In [ ]:
df32[df32.index > pd.to_datetime("2020-11-11").tz_localize(LOCALIZE_US_STOCK_MARKET)]

In [ ]:
df3 = pd.read_pickle(f"{OUTPUT_FOLDER_NAME}/210109/df_2020-11-11_2020-12-11_2m_ENG.pickle")
df3

In [ ]:
str(df.index[0].date())[2:]

In [ ]:
date_start = df.index[0]
date_start

In [ ]:
date_end = pd.Timestamp.today()
date_end

In [ ]:
date_today = pd.to_datetime(pd.Timestamp.today().date())
date_today

In [ ]:
date_end = pd.to_datetime("2021-01-01")
date_end

In [ ]:
end = pd.to_datetime(pd.Timestamp.today())
start = end - pd.Timedelta(60, "d")

In [ ]:
end

In [ ]:
start

In [ ]:
end = pd.to_datetime(pd.Timestamp.today().date()).tz_localize(LOCALIZE_US_STOCK_MARKET)
end

In [ ]:
date_start_1d = pd.to_datetime("2003-07-15")
date_today = pd.Timestamp.today().normalize()
date_start_1h = date_today - pd.Timedelta(730 - 1, "d")
date_start_2m = date_today - pd.Timedelta(60 - 1, "d")
date_start_1m = date_today - pd.Timedelta(30 - 1 , "d")
date_end = pd.to_datetime("2020-12-27")
date_end = date_today

print(f"date_start_1d={date_start_1d}")
print(f"date_start_1h={date_start_1h}")
print(f"date_start_2m={date_start_2m}")
print(f"date_start_1m={date_start_1m}")
print(f"date_today   ={date_today}")
print(f"date_end     ={date_end}")


In [ ]:
def get_list_date():
    list_date = []
    if date_end <= date_start_1d:
        print("A")
        raise RuntimeError(f"date_end={date_end} <= date_start_1d={date_start_1d}")
    elif date_end <= date_start_1h:
        print("B")
        list_date.append((date_start_1d, date_end, "1d"))
    elif date_end <= date_start_2m:
        print("C")
        list_date.append((date_start_1d, date_start_1h, "1d"))
        list_date.append((date_start_1h, date_end, "1h"))
    elif date_end <= date_start_1m:
        print("D")
        list_date.append((date_start_1d, date_start_1h, "1d"))
        list_date.append((date_start_1h, date_start_2m, "1h"))
        list_date.append((date_start_2m, date_end, "2m"))
    elif date_end <= date_today:
        print("E")
        #list_date.append((date_start_1d, date_start_1h, "1d"))
        #list_date.append((date_start_1h, date_start_2m, "1h"))
        #list_date.append((date_start_2m, date_start_1m, "2m"))
        # now comes an extra constraint that in this last 30 days interval
        # we can query for 1 minute interval in just 7 days long intervals
        # so we need to query several times (maximum 4 times)
        # first let's evaluate the boundaries for these
        date_start_1m_0 = date_start_1m + pd.Timedelta(0 * 7, "d")
        date_start_1m_1 = date_start_1m + pd.Timedelta(1 * 7, "d")
        date_start_1m_2 = date_start_1m + pd.Timedelta(2 * 7, "d")
        date_start_1m_3 = date_start_1m + pd.Timedelta(3 * 7, "d")
        date_start_1m_4 = date_start_1m + pd.Timedelta(4 * 7, "d")
        print(f"date_start_1m_0={date_start_1m_0}")
        print(f"date_start_1m_1={date_start_1m_1}")
        print(f"date_start_1m_2={date_start_1m_2}")
        print(f"date_start_1m_3={date_start_1m_3}")
        print(f"date_start_1m_4={date_start_1m_4}")
        if date_end <= date_start_1m_1:
            list_date.append((date_start_1m_0, date_end, "1m"))
        elif date_end <= date_start_1m_2:
            list_date.append((date_start_1m_0, date_start_1m_1, "1m"))
            list_date.append((date_start_1m_1, date_end, "1m"))
        elif date_end <= date_start_1m_3:
            list_date.append((date_start_1m_0, date_start_1m_1, "1m"))
            list_date.append((date_start_1m_1, date_start_1m_2, "1m"))
            list_date.append((date_start_1m_2, date_end, "1m"))
        elif date_end <= date_start_1m_4:
            list_date.append((date_start_1m_0, date_start_1m_1, "1m"))
            list_date.append((date_start_1m_1, date_start_1m_2, "1m"))
            list_date.append((date_start_1m_2, date_start_1m_3, "1m"))
            list_date.append((date_start_1m_3, date_end, "1m"))
        else:
            #list_date.append((date_start_1m_0, date_start_1m_1, "1m"))
            #list_date.append((date_start_1m_1, date_start_1m_2, "1m"))
            list_date.append((date_start_1m_2, date_start_1m_3, "1m"))
            list_date.append((date_start_1m_3, date_start_1m_4, "1m"))
            list_date.append((date_start_1m_4, date_end, "1m"))
        
    else:
        raise RuntimeError(f"date_end={date_end} > date_today={date_today}")
    #list_date.append((, , "1d"))
    return list_date
list_date = get_list_date()
list_date

In [ ]:
def get_df_2(stock_ticker, list_date, output_folder_name):
    list_df = []
    for s, e, interval in list_date:
        #string_date_start = f"20{s} 00:00:00"
        #string_date_end   = f"20{e} 00:00:00"
        #date_start = pd.to_datetime(string_date_start).tz_localize(LOCALIZE_US_STOCK_MARKET)
        #date_end = pd.to_datetime(string_date_end).tz_localize(LOCALIZE_US_STOCK_MARKET)
        date_start = s.tz_localize(LOCALIZE_US_STOCK_MARKET)
        date_end = e.tz_localize(LOCALIZE_US_STOCK_MARKET)
        logger.info(f"{stock_ticker} from {date_start} to {date_end} with interval {interval}")

        # read the data
        df = read_data(stock_ticker,
                   PERIOD,
                   date_start,
                   date_end,
                   interval,
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
        
        #
        logger.info(f"len = {len(df)}")
        if len(df) > 0:
            if stock_ticker == "AMRH":
                if interval.endswith("h") or interval.endswith("m"):
                    # ajust by the stock split of 4 stocks -> 1 stock
                    apply_split(df, 4, 1)
            # add to list
            list_df.append(df)
            # save for future
            ss = str(s.date())
            se = str(e.date())
            output_file_name = get_output_file_name(output_folder_name, ss, se, interval, stock_ticker)
            df.to_pickle(output_file_name )
                  
    # print(list_df[-1])
    return pd.concat(list_df, axis = 0)

In [ ]:
logging.info(f"{stock_ticker}")
df = get_df_2(stock_ticker, list_date, OUTPUT_FOLDER_NAME)
date_start_all = str(list_date[0][0].date())
date_end_all = str(list_date[-1][1].date())
output_file_name = get_output_file_name(OUTPUT_FOLDER_NAME, date_start_all, date_end_all, "al", stock_ticker)
df.to_pickle(output_file_name)

In [ ]:
ticker = yf.Ticker(stock_ticker)
#end = pd.to_datetime(pd.Timestamp.today().date()).tz_localize(LOCALIZE_US_STOCK_MARKET)
#start = end - pd.Timedelta(59, "d")
#end = pd.to_datetime(pd.Timestamp.today())
# end = pd.to_datetime(pd.Timestamp.today().date())#.tz_localize(LOCALIZE_US_STOCK_MARKET)
#end = pd.to_datetime(pd.Timestamp.today().date()).tz_localize("Europe/Berlin")
#start = end - pd.Timedelta(7, "d")
start = pd.to_datetime("2020-12-10 16:10:00")
end = pd.to_datetime("2020-12-10 16:15:00")
print(start)
print(end)
df = ticker.history(
        period = None,
        # start = pd.to_datetime("2021-01-01").tz_localize(LOCALIZE_US_STOCK_MARKET),
        # end = pd.to_datetime("2021-01-09").tz_localize(LOCALIZE_US_STOCK_MARKET),
        # today
        start = start,
        end = end,
        interval = "1m",
        prepost = True,
        actions = True,
        auto_adjust = True,
        )
df

In [ ]:
e=1604962800 and endTime=1610146800.

In [ ]:
np.array([1604962800, 1610146800]).astype('datetime64[s]')


In [ ]:
    datetime_end = df.index[0].tz_localize(None)
    date_short = str(datetime_end.tz_localize(None).date())[2:]

In [ ]:
# done one for some dates
if False:
    df = read_data("LAZR",
                   None,
                   pd.to_datetime("2019-03-25").tz_localize(LOCALIZE_US_STOCK_MARKET),
                   pd.to_datetime("2020-11-04").tz_localize(LOCALIZE_US_STOCK_MARKET),
                   "1h",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
df

In [ ]:
# done one for the entire period
if True:
    df = read_data(stock_ticker,
                   "max",
                   None,
                   None,
                   "1d",
                   ADD_OUTSIDE_TRADING_HOURS,
                   ADD_DIVIDENDS_AND_STOCK_SPLITS,
                   AUTO_ADJUST)
df

In [ ]:
date_start = ""

In [ ]:
plt.plot(df.Close)

In [ ]:
# example of concatenate two that changed name
if False:
    interval = "al"
    output_file_name = get_output_file_name(OUTPUT_FOLDER_NAME, "19-01-01", "20-12-31", "al", "AMRH")
    df1 = pd.read_pickle(output_file_name)
    df1

In [ ]:
if False:
    interval = "al"
    output_file_name = get_output_file_name(OUTPUT_FOLDER_NAME, "20-12-31", "21-01-01", "al", "ENVB")
    df2 = pd.read_pickle(output_file_name)
    df2

In [ ]:
if False:
    df = pd.concat([df1, df2], axis = 0)
    df

In [ ]:
if False:
    output_file_name = get_output_file_name(OUTPUT_FOLDER_NAME, "19-01-01", "21-01-01", "al", "ENVB")
    df.to_pickle(output_file_name )

In [ ]:
# sometimes buggy data with very large value in after-market, remove it
df = df[df.Close < 100000]

In [ ]:
# get only the pre-market data
get_df_pre_market(df)

In [ ]:
# get only the after-market data
get_df_after_market(df)

In [ ]:
# get only the during-market data
df2 = get_df_during_market(df)
df2

In [ ]:
# for the in-market data, add a fictious interval one minute before that ends on the open value
# so that we can plot the open value as well
df3 = add_interval_with_open(df2)
df3

In [ ]:
final_plot = plot_interactive(df)
final_plot.opts(xaxis = "bottom", title = f"Stock price of {stock_ticker}", show_legend = False) 

In [ ]:
final_plot_volume = plot_interactive_volume(df)
xaxis = None # "bottom"
final_plot_volume.opts(xaxis = xaxis, yaxis = None, title = f"Stock volume of {stock_ticker}") 